In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
import matplotlib.pyplot as plt
import scipy.io as sio
import tools.graph_processing as gp

In [2]:
path_to_labelled_graphs = '../data/Oasis_original_new_with_dummy/labelled_graphs'
path_ro_correspondence = 'gender_correspondence.pickle'

In [3]:
def create_clusters_lists_with_label_gender(list_graphs,gender_corresp,label_attribute="label_dbscan"):

    result_dict = {}
    label_depths = {}
    label_gender = {}

    for i_graph, graph in enumerate(list_graphs):
        for node in graph.nodes:
            if not graph.nodes[node]["is_dummy"]:
                label_cluster = graph.nodes[node][label_attribute]
                
                if label_cluster in result_dict:
                    
                    #retrieve depth of the corresponding label in that graph
                    depth_value = graph.nodes[node]['depth']
                    
                    result_dict[label_cluster].append((i_graph, node))
                    label_depths[label_cluster].append(depth_value)
                    label_gender[label_cluster].append(gender_corresp[i_graph])
                    
                else:
                    #retrieve depth of the corresponding label in that graph
                    depth_value = graph.nodes[node]['depth']
                    
                    result_dict[label_cluster] = [(i_graph, node)]
                    label_depths[label_cluster] = [depth_value]
                    label_gender[label_cluster] = [gender_corresp[i_graph]]


    return result_dict,label_depths,label_gender

In [4]:
labeled_graphs = gp.load_graphs_in_list(path_to_labelled_graphs)
corresp = np.array(nx.read_gpickle(path_ro_correspondence))

In [5]:
labeled_graphs[0].nodes.data()[0]

{'vertex_index': 1357,
 'sphere_3dcoords': array([  6.50138903, -93.98175049, -33.54344177]),
 'sphere_coords': [-1.5017292499542236, 1.9128626585006714],
 'basin_label': 307.0,
 'basin_area': 447.01047,
 'depth': -0.18439683,
 'basin_thickness': 2.5869646072387695,
 'ico100_7_vertex_index': 7574,
 'label_neuroimage': 28,
 'label_media': 90,
 'sphere_3dcoords_noreg': array([ 17.402021, -91.770744, -35.71135 ], dtype=float32),
 'ico100_7_vertex_index_noreg': 11492,
 'is_dummy': False,
 'labelling_kerGM': 0.6726961301647116,
 'labelling_mALS': 0.6726961301647116,
 'labelling_mSync': 0.6726961301647116,
 'labelling_CAO': 0.2269832262372122,
 'labelling_MatchEig': 0.6726961301647116}

In [6]:
result_dict,label_depths,label_gender = create_clusters_lists_with_label_gender(labeled_graphs,corresp[:,2],'labelling_mALS')

In [7]:
label_gen_sep = []
for key in label_gender.keys():
    M = []
    F = []
    for i in range(len(label_gender[key])):
        
        if label_gender[key][i] == 'F':
            
            F.append(label_depths[key][i])
        else:
            M.append(label_depths[key][i])
            
    label_gen_sep.append([M,F])
         
    

In [9]:
for 

[[[-0.93753695,
   -1.0432832,
   -0.70390266,
   -0.45560634,
   -1.1638522,
   -0.1566572,
   -0.9012098,
   -0.11415763,
   -0.78365535,
   0.13963665,
   -0.9896296,
   -0.6180877,
   0.48491007,
   -0.3571495,
   -0.9553414,
   -0.005535764,
   -0.8849734,
   -0.4598663,
   0.28231633,
   -0.2696124,
   -0.49694282],
  [-0.18439683,
   0.5117578,
   -0.8911216,
   -0.5467673,
   -0.90157324,
   -1.3031323,
   -0.57829064,
   -0.45848858,
   -0.6557367,
   0.7027702,
   -1.0530351,
   -0.58646977,
   -0.4556983,
   0.61299527,
   0.6242989,
   0.18786533,
   -0.8020224,
   -0.098019466,
   -0.009999736,
   -0.4348985,
   -0.9922196,
   -1.0880289,
   -0.24184334,
   0.42287195]],
 [[-0.12659615,
   0.18049002,
   0.22486097,
   -0.02675764,
   0.87198114,
   -0.04307983,
   1.0408891,
   0.09548654,
   0.56341136,
   -0.1779305,
   0.31481138,
   -0.047117025,
   -0.2530798,
   0.1381937,
   0.35646987,
   0.32695088,
   -0.86873615,
   -0.6246846,
   0.34713167,
   -0.28407496,
  

In [ ]:
# 1st list M, 2nd F
t_stats = []

for lst in label_gen_sep:
    
    res = stats.ttest_ind(a=lst[0], b=lst[1], equal_var=True)
    
    print(res)

In [ ]:
type(res.statistic)

In [ ]:
res

In [ ]:
label_gen_sep[-1]